In [1]:
import pandas as pd
import numpy as np
import time

train_pca=pd.read_csv("C:\\Users\\98gud\\Desktop\\paper\\ieee-fraud-detection\\train_pcaa.csv")
train=pd.read_csv("C:\\Users\\98gud\\Desktop\\paper\\ieee-fraud-detection\\train.csv")
train_y=pd.read_csv("C:\\Users\\98gud\\Desktop\\paper\\ieee-fraud-detection\\train_y.csv")

In [2]:
scores={}
right_index={}

## description

isFraud = { '0' : 'Normal' , '1' : 'Fraud' }  
predited value = { '0' or '1' : 'Normal' , '-1' : 'Predicted Fraud'}  

iforest score: lower means outlier  
lof score: lower means outlier  
hdbscan : higher means outlier  
mahalanobis : higher means outlier  

In [3]:
## Isolation forest(PCA)

from sklearn.ensemble import IsolationForest

iforest=IsolationForest(random_state=1022)
iforest.fit(train_pca)
predict=iforest.predict(train_pca)

train_y['iforest']=predict

scores['iforest']=iforest.score_samples(train_pca)
right_index['iforest']=train_y.loc[(train_y['isFraud']==1)&(train_y['iforest']==-1)].index
pd.crosstab(train_y['iforest'],train_y['isFraud'])

isFraud,0,1
iforest,,
-1,34093,3323
1,535784,17340


In [4]:
#Local Outlier Factor(PCA)

from sklearn.neighbors import LocalOutlierFactor

lof=LocalOutlierFactor(n_neighbors=5)
predict=lof.fit_predict(train_pca)

train_y['lof']=predict

scores['lof']=lof.negative_outlier_factor_
right_index['loc']=train_y.loc[(train_y['isFraud']==1)&(train_y['lof']==-1)].index
pd.crosstab(train_y['lof'],train_y['isFraud'])

isFraud,0,1
lof,,
-1,62063,3283
1,507814,17380


In [5]:
## Iforest+LOF(by union)

predict=np.zeros(train.shape[0])
for idx in train_y.loc[(train_y['iforest']==-1) | (train_y['lof']==-1)].index:
    predict[idx]=-1
train_y['iforest+lof_u']=predict
right_index['iforest+lof_u']=train_y.loc[(train_y['isFraud']==1)&(train_y['iforest+lof_u']==-1)].index
pd.crosstab(train_y['iforest+lof_u'],train_y['isFraud'])

isFraud,0,1
iforest+lof_u,,
-1.0,90842,6179
0.0,479035,14484


In [6]:
## Iforest+LOF(by inter)

predict=np.zeros(train.shape[0])
for idx in train_y.loc[(train_y['iforest']==-1) & (train_y['lof']==-1)].index:
    predict[idx]=-1
train_y['iforest+lof_i']=predict
right_index['iforest+lof_i']=train_y.loc[(train_y['isFraud']==1) & (train_y['iforest+lof_i']==-1)].index
pd.crosstab(train_y['iforest+lof_i'],train_y['isFraud'])

isFraud,0,1
iforest+lof_i,,
-1.0,5314,427
0.0,564563,20236


In [7]:
import hdbscan

cluster=hdbscan.HDBSCAN().fit(train_pca)

scores['hdbscan']=cluster.outlier_scores_
threshold=pd.Series(scores['hdbscan']).quantile(0.9)

outliers=np.where(scores['hdbscan']>threshold)[0]
right_index['hdbscan']=set(train_y.loc[train_y['isFraud']==1].index).intersection(set(outliers))

hdbscan=np.zeros(train_pca.shape[0])
for idx in outliers:
    hdbscan[idx]=-1

train_y['hdbscan']=hdbscan

pd.crosstab(train_y['hdbscan'],train_y['isFraud'])

isFraud,0,1
hdbscan,,
-1.0,56395,2659
0.0,513482,18004


In [8]:
from scipy.stats import chi2

covariance=np.cov(train_pca,rowvar=False)
covariance_reverse=np.linalg.matrix_power(covariance,-1)

centerpoint=np.mean(train_pca,axis=0)

distances=[]
for i,val in enumerate(train_pca.values):
    x1=val
    x2=centerpoint
    distance=(x1-x2).T.dot(covariance_reverse).dot(x1-x2)
    distances.append(distance)
    
distances=np.array(distances)

cutoff=chi2.ppf(0.8,train_pca.shape[1])
outlier_ma=np.where(distances>cutoff)[0]


mahalanobis=np.zeros(train_pca.shape[0])
for idx in outlier_ma:
    mahalanobis[idx]=-1
    
train_y['mahalanobis']=mahalanobis
scores['mahalanobis']=distances
right_index['mahalanobis']=train_y.loc[(train_y['isFraud']==1) & (train_y['mahalanobis']==-1)].index

pd.crosstab(train_y['mahalanobis'],train_y['isFraud'])

isFraud,0,1
mahalanobis,,
-1.0,7278,657
0.0,562599,20006


In [9]:
train_y

,isFraud,iforest,lof,iforest+lof_u,iforest+lof_i,hdbscan,mahalanobis
0,0,1,1,0.0,0.0,0.0,0.0
1,0,1,1,0.0,0.0,0.0,0.0
2,0,1,1,0.0,0.0,0.0,0.0
3,0,-1,1,-1.0,0.0,0.0,0.0
4,0,1,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
590535,0,1,-1,-1.0,0.0,-1.0,0.0
590536,0,1,1,0.0,0.0,0.0,0.0
590537,0,1,1,0.0,0.0,0.0,0.0
590538,0,-1,1,-1.0,0.0,0.0,0.0


In [19]:
columns=train_y.columns
for i in columns:
    if i in ['isFraud','iforest+lof_u','iforest+lof_i']: pass
    else: 
        train_y[i+" scores"]=scores[i]
train_y

,isFraud,iforest,lof,iforest+lof_u,iforest+lof_i,hdbscan,mahalanobis,iforest scores,lof scores,hdbscan scores,mahalanobis scores
0,0,1,1,0.0,0.0,0.0,0.0,-0.391002,-0.997672,0.015392,0.068867
1,0,1,1,0.0,0.0,0.0,0.0,-0.299100,-1.008481,0.013761,0.011401
2,0,1,1,0.0,0.0,0.0,0.0,-0.299100,-1.207288,0.003123,0.011401
3,0,-1,1,-1.0,0.0,0.0,0.0,-0.642361,-1.085168,0.007088,1.166482
4,0,1,1,0.0,0.0,0.0,0.0,-0.299100,-1.089259,0.091146,0.011401
...,...,...,...,...,...,...,...,...,...,...,...
590535,0,1,-1,-1.0,0.0,-1.0,0.0,-0.303563,-2.675082,0.807587,0.010451
590536,0,1,1,0.0,0.0,0.0,0.0,-0.299100,-1.407753,0.059276,0.011401
590537,0,1,1,0.0,0.0,0.0,0.0,-0.299100,-1.157963,0.023822,0.011401
590538,0,-1,1,-1.0,0.0,0.0,0.0,-0.699328,-1.103259,0.203412,4.066957


In [20]:
train_y.to_csv("C:\\Users\\98gud\\Desktop\\outlier_detection\\train_scores.csv",index=False)